In [1]:
weather_data_dict = {
    "Outlook": ["sunny", "sunny", "overcast", "rain", "rain", "rain", "overcast",
                "sunny", "sunny", "rain", "sunny", "overcast", "overcast", "rain"],
    "Temperature": ["hot", "hot", "hot", "mild", "cool", "cool", "cool",
                    "mild", "cool", "mild", "mild", "mild", "hot", "mild"],
    "Humidity": ["high", "high", "high", "high", "normal", "normal", "normal",
                 "high", "normal", "normal", "normal", "high", "normal", "high"],
    "Windy": [False, True, False, False, False, True, True,
              False, False, False, True, True, False, True],
    "Class": ["N", "N", "P", "P", "P", "N", "P",
              "N", "P", "P", "P", "P", "P", "N"]
}


In [ ]:
activity_data_dict = {
    "Deadline": ["Urgent", "Urgent", "Near", "None", "None", "None", "Near", "Near", "Near", "Urgent"],
    "Is there a party?": ["Yes", "No", "Yes", "Yes", "No", "Yes", "No", "No", "Yes", "No"],
    "Lazy?": ["Yes", "Yes", "Yes", "No", "No", "No", "No", "Yes", "Yes", "No"],
    "Activity": ["Party", "Study", "Party", "Party", "Pub", "Party", "Study", "TV", "Party", "Study"]
}


In [ ]:
import pandas as pd

weather_data = pd.DataFrame(weather_data_dict)
activity_data = pd.DataFrame(activity_data_dict)

print("Weather Data:")
print(weather_data)

print("\nActivity Data:")
print(activity_data)


Weather Data:
     Outlook Temperature Humidity  Windy Class
0      sunny         hot     high  False     N
1      sunny         hot     high   True     N
2   overcast         hot     high  False     P
3       rain        mild     high  False     P
4       rain        cool   normal  False     P
5       rain        cool   normal   True     N
6   overcast        cool   normal   True     P
7      sunny        mild     high  False     N
8      sunny        cool   normal  False     P
9       rain        mild   normal  False     P
10     sunny        mild   normal   True     P
11  overcast        mild     high   True     P
12  overcast         hot   normal  False     P
13      rain        mild     high   True     N

Activity Data:
  Deadline Is there a party? Lazy? Activity
0   Urgent               Yes   Yes    Party
1   Urgent                No   Yes    Study
2     Near               Yes   Yes    Party
3     None               Yes    No    Party
4     None                No    No      Pub
5

In [ ]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

le_outlook = LabelEncoder()
le_temperature = LabelEncoder()
le_humidity = LabelEncoder()
le_windy = LabelEncoder()
le_class = LabelEncoder()

weather_data_encoded = weather_data.copy()
weather_data_encoded["Outlook"] = le_outlook.fit_transform(weather_data["Outlook"])
weather_data_encoded["Temperature"] = le_temperature.fit_transform(weather_data["Temperature"])
weather_data_encoded["Humidity"] = le_humidity.fit_transform(weather_data["Humidity"])
weather_data_encoded["Windy"] = le_windy.fit_transform(weather_data["Windy"])
weather_data_encoded["Class"] = le_class.fit_transform(weather_data["Class"])


X_weather = weather_data_encoded.drop("Class", axis=1)
y_weather = weather_data_encoded["Class"]
X_train_weather, X_test_weather, y_train_weather, y_test_weather = train_test_split(X_weather, y_weather, test_size=0.3, random_state=42)

nb_weather = CategoricalNB()
nb_weather.fit(X_train_weather, y_train_weather)

y_pred_weather = nb_weather.predict(X_test_weather)


y_train_pred_weather = nb_weather.predict(X_train_weather)
y_test_pred_weather = nb_weather.predict(X_test_weather)
train_accuracy_sklearn = accuracy_score(y_train_weather, y_train_pred_weather)
test_accuracy_sklearn = accuracy_score(y_test_weather, y_test_pred_weather)

print("Scikit-learn Naive Bayes Model Results:")
print("Training Accuracy:", train_accuracy_sklearn)
print("Test Accuracy:", test_accuracy_sklearn)
print("\nClassification Report for Test Data:")
print(classification_report(y_test_weather, y_test_pred_weather))


Scikit-learn Naive Bayes Model Results:
Training Accuracy: 0.8888888888888888
Test Accuracy: 0.6

Classification Report for Test Data:
              precision    recall  f1-score   support

           0       0.50      0.50      0.50         2
           1       0.67      0.67      0.67         3

    accuracy                           0.60         5
   macro avg       0.58      0.58      0.58         5
weighted avg       0.60      0.60      0.60         5



In [ ]:
test_data = {
    'Outlook': ['sunny'],
    'Temperature': ['cool'],
    'Humidity': ['normal'],
    'Windy': ['False']
}

test_df = pd.DataFrame(test_data)

test_df_encoded = test_df.copy()
test_df_encoded["Outlook"] = le_outlook.transform(test_df["Outlook"])
test_df_encoded["Temperature"] = le_temperature.transform(test_df["Temperature"])
test_df_encoded["Humidity"] = le_humidity.transform(test_df["Humidity"])
test_df_encoded["Windy"] = le_windy.transform(test_df["Windy"])

X_test_single = test_df_encoded.drop("Class", axis=1, errors="ignore")

prediction = nb_weather.predict(X_test_single)

predicted_class = le_class.inverse_transform(prediction)

print(f"Prediction for the input row: {predicted_class[0]}")


Prediction for the input row: P


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

class NaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_prior = {c: np.mean(y == c) for c in self.classes}
        self.feature_prob = {}

        for c in self.classes:
            class_data = X[y == c]
            self.feature_prob[c] = {
                col: class_data[col].value_counts(normalize=True).to_dict()
                for col in X.columns
            }

    def predict(self, X):
        predictions = []
        for _, row in X.iterrows():
            class_probs = {}
            for c in self.classes:
                prob = self.class_prior[c]
                for col in X.columns:
                    feature_value = row[col]
                    prob *= self.feature_prob[c][col].get(feature_value, 1e-6)
                class_probs[c] = prob
            predictions.append(max(class_probs, key=class_probs.get))
        return np.array(predictions)

le_outlook = LabelEncoder()
le_temperature = LabelEncoder()
le_humidity = LabelEncoder()
le_windy = LabelEncoder()
le_class = LabelEncoder()

weather_data_encoded = weather_data.copy()
weather_data_encoded["Outlook"] = le_outlook.fit_transform(weather_data["Outlook"])
weather_data_encoded["Temperature"] = le_temperature.fit_transform(weather_data["Temperature"])
weather_data_encoded["Humidity"] = le_humidity.fit_transform(weather_data["Humidity"])
weather_data_encoded["Windy"] = le_windy.fit_transform(weather_data["Windy"])
weather_data_encoded["Class"] = le_class.fit_transform(weather_data["Class"])

X_weather = weather_data_encoded.drop("Class", axis=1)
y_weather = weather_data_encoded["Class"]
X_train_weather, X_test_weather, y_train_weather, y_test_weather = train_test_split(
    X_weather, y_weather, test_size=0.3, random_state=42)

nb_custom = NaiveBayes()
nb_custom.fit(X_train_weather, y_train_weather)

y_pred_custom = nb_custom.predict(X_test_weather)

y_train_pred_custom = nb_custom.predict(X_train_weather)
y_test_pred_custom = nb_custom.predict(X_test_weather)

train_accuracy_custom = accuracy_score(y_train_weather, y_train_pred_custom)
test_accuracy_custom = accuracy_score(y_test_weather, y_test_pred_custom)

print("Custom Naive Bayes Model Results:")
print("Training Accuracy:", train_accuracy_custom)
print("Test Accuracy:", test_accuracy_custom)
print("\nClassification Report for Test Data:")
print(classification_report(y_test_weather, y_test_pred_custom))


Custom Naive Bayes Model Results:
Training Accuracy: 0.8888888888888888
Test Accuracy: 0.8

Classification Report for Test Data:
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.75      1.00      0.86         3

    accuracy                           0.80         5
   macro avg       0.88      0.75      0.76         5
weighted avg       0.85      0.80      0.78         5



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, classification_report

# Define the dataset
data = {
    "Deadline": ["Urgent", "Urgent", "Near", "None", "None", "None", "Near", "Near", "Near", "Urgent"],
    "Is there a party?": ["Yes", "No", "Yes", "Yes", "No", "Yes", "No", "No", "Yes", "No"],
    "Lazy?": ["Yes", "Yes", "Yes", "No", "Yes", "Yes", "No", "Yes", "Yes", "No"],
    "Activity": ["Party", "Study", "Party", "Party", "Pub", "Party", "Study", "TV", "Party", "Study"]
}

df = pd.DataFrame(data)

# Encode categorical features
le_deadline = LabelEncoder()
le_party = LabelEncoder()
le_lazy = LabelEncoder()
le_activity = LabelEncoder()

df["Deadline"] = le_deadline.fit_transform(df["Deadline"])
df["Is there a party?"] = le_party.fit_transform(df["Is there a party?"])
df["Lazy?"] = le_lazy.fit_transform(df["Lazy?"])
df["Activity"] = le_activity.fit_transform(df["Activity"])

# Split dataset
X = df.drop("Activity", axis=1)
y = df["Activity"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Approach 1: Using Scikit-learn's Naive Bayes
nb_sklearn = CategoricalNB()
nb_sklearn.fit(X_train, y_train)
y_train_pred_sklearn = nb_sklearn.predict(X_train)
y_pred_sklearn = nb_sklearn.predict(X_test)

print("Scikit-learn Naive Bayes Results:")
print("Train Accuracy:", accuracy_score(y_train, y_train_pred_sklearn))
print("Test Accuracy:", accuracy_score(y_test, y_pred_sklearn))
print(classification_report(y_test, y_pred_sklearn))

# Approach 2: Implementing Naive Bayes from scratch
class NaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_prior = {c: np.mean(y == c) for c in self.classes}
        self.feature_prob = {}

        for c in self.classes:
            class_data = X[y == c]
            self.feature_prob[c] = {
                col: class_data[col].value_counts(normalize=True).to_dict()
                for col in X.columns
            }

    def predict(self, X):
        predictions = []
        for _, row in X.iterrows():
            class_probs = {}
            for c in self.classes:
                prob = self.class_prior[c]
                for col in X.columns:
                    feature_value = row[col]
                    prob *= self.feature_prob[c][col].get(feature_value, 1e-6)
                class_probs[c] = prob
            predictions.append(max(class_probs, key=class_probs.get))
        return np.array(predictions)

# Train custom Naive Bayes classifier
nb_custom = NaiveBayes()
nb_custom.fit(X_train, y_train)
y_train_pred_custom = nb_custom.predict(X_train)
y_pred_custom = nb_custom.predict(X_test)

print("Custom Naive Bayes Results:")
print("Train Accuracy:", accuracy_score(y_train, y_train_pred_custom))
print("Test Accuracy:", accuracy_score(y_test, y_pred_custom))
print(classification_report(y_test, y_pred_custom))

# Test Case
sample_test = pd.DataFrame({
    "Deadline": [le_deadline.transform(["Near"])[0]],
    "Is there a party?": [le_party.transform(["Yes"])[0]],
    "Lazy?": [le_lazy.transform(["Yes"])[0]]
})

print("Sample Test Case Prediction (Sklearn):", le_activity.inverse_transform(nb_sklearn.predict(sample_test)))
print("Sample Test Case Prediction (Custom):", le_activity.inverse_transform(nb_custom.predict(sample_test)))

Scikit-learn Naive Bayes Results:
Train Accuracy: 1.0
Test Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

Custom Naive Bayes Results:
Train Accuracy: 1.0
Test Accuracy: 0.6666666666666666
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         1

    accuracy                           0.67         3
   macro avg       0.33      0.33      0.33         3
weighted avg       0.67      0.67      0.67         3

Sample Test Case Prediction (Sklearn): ['Party']
Sample Test Case Prediction (Custom): ['Party']


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_